In [2]:
%load_ext autoreload
%autoreload 2
import numpy as np
import xarray as xr
from matplotlib import pyplot as plt
import pandas as pd
import sys
# sys.path.append('/uni-mainz.de/homes/odoensen/Work/Scripts/')
sys.path.append('/home/onno/Thesis/Scripts')
import my_tools
from my_tools import plot_dic, file_dic
import matplotlib.ticker as mtick
from glob import glob
import matplotlib

%matplotlib qt

In [13]:
#set path
# path = '/uni-mainz.de/homes/odoensen/Work/T850/HK/'
path = '/media/onno/Algemeen/Thesis/GFS_T850/HK/'
path_count = '/media/onno/Algemeen/Thesis/GFS_T850/GSS/'

Overview of mean HK in all regions of Europe


In [8]:
import matplotlib.gridspec as gridspec
import matplotlib

coordinatez = [
(62,54,0,8),#Germany
(54,46,0,8), #Spain
(46,38,0,8), #Ukraine
(62,54,8,16),#Germany
(54,46,8,16), #Spain
(46,38,8,16), #Ukraine
(62,54,16,24),#Germany
(54,46,16,24), #Spain
(46,38,16,24)] #Ukraine

seasonz = {'MAM':[3,4,5],
          'JJA':[6,7,8],
          'SON':[9,10,11],
          'DJF':[12,1,2]}

seasonz_list = ['DJF','MAM','JJA','SON',
               'DJF','MAM','JJA','SON',
               'DJF','MAM','JJA','SON',
               'DJF','MAM','JJA','SON']

extremez_list = ['persistent_hw','persistent_hw','persistent_hw','persistent_hw',
                'persistent_cw','persistent_cw','persistent_cw','persistent_cw']



colorz = [(0,'#ff0000'),(0.25,'#ff6600'),(0.5,'#ffff00'),
          (0.7,'#33cc33'),(1,'#006600')]   
cmap = matplotlib.colors.LinearSegmentedColormap.from_list('custom', colorz, N=1024)

lead_dayz = [1,3,5,7,9]
fcst_modelz = ['GFS','ERA5RF']
file = 'HK_lead_day_{}_lon_{}_{}_lat_{}_{}_{}.csv'
for fcst_model in fcst_modelz:
    for lead_day in lead_dayz:
        fig,axz = plt.subplots(3,3,figsize=(16,11))
        fig.suptitle('Gilbert Skill Score per Season and per Region. Day {} Forecast {} '.format(lead_day,fcst_model))
        for i,ax in enumerate(axz.flat):
            lat_0,lat_1,lon_0,lon_1 = coordinatez[i]
            df = pd.read_csv(path+file.format(lead_day,lon_0,lon_1,
                                              lat_1,lat_0,fcst_model),
                            index_col=0)
            df.index = pd.to_datetime(df.index)
            inner_grid = gridspec.GridSpecFromSubplotSpec(2,4,subplot_spec=ax,
                                                         hspace=0,wspace=0)
            ax.set_yticks([0.25,0.75])
            ax.set_xticks([0.125,0.375,0.625,0.875])
            ax.set_xticklabels(['DJF','MAM','JJA','SON'])
            ax.set_yticklabels(['PH','PC'][::-1])
            ax.set_title('{}-{} °N {}-{} °E'.format(
                                                    lat_1,lat_0,
                                                    lon_0,lon_1))
            for j,inner in enumerate(inner_grid):
                df_sub = df[extremez_list[j]].dropna()
                df_sub_season = df_sub[np.isin(df_sub.index.month,seasonz[seasonz_list[j]])]
                HK_mean_season = df_sub_season.mean()
                ax1 = fig.add_subplot(inner)
                ax1.set_xticklabels([])
                ax1.set_yticklabels([])
                ax1.set_xticks([])
                ax1.set_yticks([])
                ax1.text(0.5,0.5,'{:.2f}'.format(HK_mean_season),
                         horizontalalignment='center',
                        verticalalignment='center',fontsize=15)
                ax1.set_facecolor(cmap(HK_mean_season))
        fig.savefig(path+'figures/total_HK_lead_day_{}_{}_3_by_3'.format(lead_day,fcst_model),dpi=150)
        plt.close(fig)
#             ax.set_aspect('equal')


In [14]:
equal_weight=False
lead_dayz = [1,3,5,7,9]
coordinatez = [
(62,54,0,8),#Germany
(54,46,0,8), #Spain
(46,38,0,8), #Ukraine
(62,54,8,16),#Germany
(54,46,8,16), #Spain
(46,38,8,16), #Ukraine
(62,54,16,24),#Germany
(54,46,16,24), #Spain
(46,38,16,24)] #Ukraine
seasonz = {'MAM':[3,4,5],
          'JJA':[6,7,8],
          'SON':[9,10,11],
          'DJF':[12,1,2]}

modelz = ['GFS','ERA5RF']
index_list = ['DJF','MAM','JJA','SON']

seasonz_list = ['DJF','MAM','JJA','SON',
               'DJF','MAM','JJA','SON',
               'DJF','MAM','JJA','SON',
               'DJF','MAM','JJA','SON']

extremez_list = ['persistent_hw','persistent_hw','persistent_hw','persistent_hw',
                'persistent_cw','persistent_cw','persistent_cw','persistent_cw']
colorz = [(0,'#ff0000'),(0.25,'#ff6600'),(0.5,'#ffff00'),
          (0.7,'#33cc33'),(1,'#006600')]   
cmap = matplotlib.colors.LinearSegmentedColormap.from_list('custom', colorz, N=1024)
tickz_y = np.arange(4)
tickz_x = np.arange(2)


for lead_day in lead_dayz:
    meanz = []
    for model in modelz:
        if equal_weight:
            dfs_mean = []

#             filez_mean = sorted(glob(path+'Monthly_mean_HK_lead_day_{}_*_{}.csv'.format(lead_day,model)))
            for lat_0,lat_1,lon_0,lon_1 in coordinatez:
                file_mean = 'Monthly_mean_HK_lead_day_{}_lon_{}_{}_lat_{}_{}_{}.csv'.format(lead_day,lon_0,lon_1,
                                                                                            lat_1,lat_0,model)
                df = pd.read_csv(path+file_mean, index_col=0, header=0)
                dfs_mean.append(df)
            df_concat = pd.concat(dfs_mean)
            df_mean = df_concat.groupby(df_concat.index).mean()
            df_mean = df_mean.reindex(index=index_list)
            meanz.append(df_mean)
        else:
            dfs_count = []
#             filez_count = glob(path+'Count_season*.csv')
            for lat_0,lat_1,lon_0,lon_1 in coordinatez:
                file_count = 'Count_season_lon_{}_{}_lat_{}_{}.csv'.format(lon_0,lon_1,lat_1,lat_0)
                df = pd.read_csv(path_count+file_count,index_col=0, header=0)
                dfs_count.append(df)
            df_concat = pd.concat(dfs_count)
            df_count_sum = df_concat.groupby(df_concat.index).sum()
            dfs_mean = []
#             filez_mean = sorted(glob(path+'Monthly_mean_HK_lead_day_{}_*_{}.csv'.format(lead_day,model)))
#             filez_count = sorted(glob(path+'Count_season*.csv'.format(lead_day)))
            for lat_0,lat_1,lon_0,lon_1 in coordinatez:
                file_mean = 'Monthly_mean_HK_lead_day_{}_lon_{}_{}_lat_{}_{}_{}.csv'.format(lead_day,lon_0,lon_1,
                                                                                            lat_1,lat_0,model)
                file_count = 'Count_season_lon_{}_{}_lat_{}_{}.csv'.format(lon_0,lon_1,lat_1,lat_0)

                df = pd.read_csv(path+file_mean, index_col=0, header=0)
                df_count = pd.read_csv(path_count+file_count, index_col=0, header=0)
                weight = df_count/df_count_sum
                df_weighted = df * weight
                dfs_mean.append(df_weighted)
            df_concat = pd.concat(dfs_mean)
            df_weighted_mean = df_concat.groupby(df_concat.index).sum()
            df_weighted_mean = df_weighted_mean.reindex(index=index_list)
            meanz.append(df_weighted_mean)
    fig,axz = plt.subplots(1,2,figsize=(12,6))
    for i,ax in enumerate(axz):
        ax.imshow(meanz[i].values,cmap = cmap,
                 vmin=0,vmax=1,aspect='auto')
        ax.set_xticks(tickz_x)
        ax.set_yticks(tickz_y)
        ax.set_yticklabels(index_list)
        ax.set_xticklabels(['Heat','Cold'])
        ax.set_title(modelz[i])
        for y in range(meanz[i].shape[0]):
            for x in range(meanz[i].shape[1]):
                ax.text(x,y,'{:.2f}'.format(meanz[i].values[y,x]),
                       ha="center", va="center", color="k",fontsize=15)
    if equal_weight:
        fig.suptitle('Forecast Day {} Equal Weight Mean'.format(lead_day))
        fig.savefig(path+'figures/HK_lead_day_{}_equal_weights_3by3.jpg'.format(lead_day))
    else:
        fig.suptitle('Forecast Day {} Weighted Mean'.format(lead_day))
        fig.savefig(path+'figures/HK_lead_day_{}_weighted_3by3.jpg'.format(lead_day))
    plt.close(fig)

    

        

In [15]:
plt.close('all')

In [15]:
import matplotlib.gridspec as gridspec
import matplotlib

coordinatez = [
(62,54,0,8),#Germany
(54,46,0,8), #Spain
(46,38,0,8), #Ukraine
(62,54,8,16),#Germany
(54,46,8,16), #Spain
(46,38,8,16), #Ukraine
(62,54,16,24),#Germany
(54,46,16,24), #Spain
(46,38,16,24)] #Ukraine

seasonz = {'MAM':[3,4,5],
          'JJA':[6,7,8],
          'SON':[9,10,11],
          'DJF':[12,1,2]}

seasonz_list = ['DJF','MAM','JJA','SON',
               'DJF','MAM','JJA','SON',
               'DJF','MAM','JJA','SON',
               'DJF','MAM','JJA','SON']

extremez_list = ['persistent_hw','persistent_hw','persistent_hw','persistent_hw',
                'persistent_cw','persistent_cw','persistent_cw','persistent_cw']



colorz = [(0,'#ff0000'),(0.25,'#ff6600'),(0.5,'#ffff00'),
          (0.7,'#33cc33'),(1,'#006600')]   
cmap = matplotlib.colors.LinearSegmentedColormap.from_list('custom', colorz, N=1024)

lead_dayz = [1,3,5,7,9]
fcst_modelz = ['GFS','ERA5RF']
file = 'HK_lead_day_{}_lon_{}_{}_lat_{}_{}_{}.csv'
for fcst_model in fcst_modelz:
    for lead_day in lead_dayz:
        fig,axz = plt.subplots(2,3,figsize=(16,11))
        fig.suptitle('Gilbert Skill Score per Season and per Region. Day {} Forecast {} '.format(lead_day,fcst_model))
        for i,ax in enumerate(axz.flat):
            lat_0,lat_1,lon_0,lon_1 = coordinatez[i]
            df = pd.read_csv(path+file.format(lead_day,lon_0,lon_1,
                                              lat_1,lat_0,fcst_model),
                            index_col=0)
            sys.exit()
            df.index = pd.to_datetime(df.index)
            inner_grid = gridspec.GridSpecFromSubplotSpec(2,4,subplot_spec=ax,
                                                         hspace=0,wspace=0)
            ax.set_yticks([0.25,0.75])
            ax.set_xticks([0.125,0.375,0.625,0.875])
            ax.set_xticklabels(['DJF','MAM','JJA','SON'])
            ax.set_yticklabels(['PH','PC'][::-1])
            ax.set_title('{}-{} °N {}-{} °E'.format(
                                                    lat_1,lat_0,
                                                    lon_0,lon_1))
            for j,inner in enumerate(inner_grid):
                df_sub = df[extremez_list[j]].dropna()
                df_sub_season = df_sub[np.isin(df_sub.index.month,seasonz[seasonz_list[j]])]
                HK_mean_season = df_sub_season.mean()
                ax1 = fig.add_subplot(inner)
                ax1.set_xticklabels([])
                ax1.set_yticklabels([])
                ax1.set_xticks([])
                ax1.set_yticks([])
                ax1.text(0.5,0.5,'{:.2f}'.format(HK_mean_season),
                         horizontalalignment='center',
                        verticalalignment='center',fontsize=15)
                ax1.set_facecolor(cmap(HK_mean_season))
        fig.savefig(path+'figures/total_HK_lead_day_{}_{}_3_by_3'.format(lead_day,fcst_model),dpi=150)
        plt.close(fig)

SystemExit: 